In [15]:
import cv2
import numpy as np
import time
import pyttsx3
from threading import Thread

class WorkoutChatbot:
    def __init__(self): 
        print("Initializing WorkoutChatbot...")
 
        self.engine = pyttsx3.init()
        self.engine.setProperty('rate', 150)
      
        print("Attempting to initialize camera...")
        self.cap = None
        try:
            self.cap = cv2.VideoCapture(0) 
            if not self.cap.isOpened():
                raise Exception("Camera failed to open")
            print("Camera initialized successfully")
        except Exception as e:
            print(f"Camera initialization failed: {str(e)}")
            raise
            
        self.is_running = False
        self.exercise_type = ""
        self.target_reps = 0
        self.current_reps = 0
        self.prev_position = None

    def speak(self, text):
        """Thread-safe text-to-speech function"""
        def speak_thread():
            try:
                self.engine.say(text)
                self.engine.runAndWait()
            except Exception as e:
                print(f"Speech error: {str(e)}")
        Thread(target=speak_thread).start()

    def get_user_input(self):
        """Get exercise type and target reps from user"""
        self.exercise_type = input("What exercise are you doing? (e.g., pushups, squats): ")
        while True:
            try:
                self.target_reps = int(input("How many reps do you want to do? "))
                if self.target_reps > 0:
                    break
                print("Please enter a positive number!")
            except ValueError:
                print("Please enter a valid number!")
        
        self.speak(f"Let's do {self.target_reps} {self.exercise_type}! Start whenever you're ready!")

    def detect_movement(self, frame):
        """Simple movement detection"""
        try:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (21, 21), 0)
            height, width = blurred.shape
            roi = blurred[int(height*0.3):int(height*0.7), int(width*0.3):int(width*0.7)]
            return np.mean(roi)
        except Exception as e:
            print(f"Movement detection error: {str(e)}")
            return None

    def count_reps(self):
        """Main rep counting logic"""
        print("Starting rep counting...")
        if self.cap is None or not self.cap.isOpened():
            print("Error: Camera not accessible")
            self.speak("Camera not available. Please check your connection.")
            return

        self.is_running = True
        last_rep_time = time.time()
        encouragement_count = 0

        while self.is_running and self.current_reps < self.target_reps:
            ret, frame = self.cap.read()
            if not ret:
                print("Error: Failed to capture frame")
                break

            current_position = self.detect_movement(frame)
            if current_position is None:
                continue
                
            if self.prev_position is not None:
                diff = abs(current_position - self.prev_position)
                if diff > 10:  # Movement detection threshold
                    if time.time() - last_rep_time > 1:  # Debounce period
                        self.current_reps += 1
                        last_rep_time = time.time()
                        self.speak(f"Rep {self.current_reps}")
                        
                        if self.current_reps == self.target_reps // 2:
                            self.speak("Halfway there, keep going!")
                        elif self.current_reps == self.target_reps - 1:
                            self.speak("Last one, you got this!")

            self.prev_position = current_position

            if time.time() - last_rep_time > 5 and self.current_reps < self.target_reps:
                encouragement_count += 1
                if encouragement_count % 2 == 0:
                    self.speak("Don't give up! You can do it!")
                else:
                    self.speak("Keep pushing, you're doing great!")

            cv2.putText(frame, f"Reps: {self.current_reps}/{self.target_reps}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Workout Counter", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cleanup()

    def cleanup(self):
        """Cleanup resources"""
        print("Cleaning up...")
        self.is_running = False
        if hasattr(self, 'cap') and self.cap is not None:
            if self.cap.isOpened():
                self.cap.release()
        cv2.destroyAllWindows()
        
        if self.current_reps >= self.target_reps:
            self.speak(f"Congratulations! You completed {self.target_reps} {self.exercise_type}!")
        else:
            self.speak(f"Great effort! You completed {self.current_reps} {self.exercise_type}!")

    def run(self):
        """Main execution method"""
        print("Welcome to your Workout Assistant!")
        self.get_user_input()
        self.count_reps()

if __name__ == "__main__":  
    try:
        chatbot = WorkoutChatbot()
        chatbot.run()
    except Exception as e:
        print(f"Main execution error: {str(e)}")

Initializing WorkoutChatbot...
Attempting to initialize camera...
Camera initialized successfully
Welcome to your Workout Assistant!


What exercise are you doing? (e.g., pushups, squats):  pushups
How many reps do you want to do?  3


Speech error: run loop already startedStarting rep counting...

Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
Speech error: run loop already started
